In [1]:
import numpy as np
import pandas as pd
import scipy
from math import sqrt
import matplotlib.pyplot as plt

In [2]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn import linear_model

In [3]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.model_selection import cross_val_score

In [4]:
from sklearn.model_selection import train_test_split

In [5]:
from sklearn.preprocessing import StandardScaler

In [6]:
rawData = pd.read_csv('default of credit card clients.csv', header=1)

In [7]:
df = rawData.rename(columns={'default payment next month': 'Default'})

In [8]:
df['PayBill1']=df['BILL_AMT1']*df['PAY_0']
df['PayBill2']=df['BILL_AMT2']*df['PAY_2']
df['PayBill3']=df['BILL_AMT3']*df['PAY_3']
df['PayBill4']=df['BILL_AMT4']*df['PAY_4']
df['PayBill5']=df['BILL_AMT5']*df['PAY_5']
df['PayBill6']=df['BILL_AMT6']*df['PAY_6']

In [9]:
df['PayPay1']=df['PAY_AMT1']*df['PAY_0']
df['PayPay2']=df['PAY_AMT2']*df['PAY_2']
df['PayPay3']=df['PAY_AMT3']*df['PAY_3']
df['PayPay4']=df['PAY_AMT4']*df['PAY_4']
df['PayPay5']=df['PAY_AMT5']*df['PAY_5']
df['PayPay6']=df['PAY_AMT6']*df['PAY_6']

In [10]:
df[['LIMIT_BAL','MARRIAGE','EDUCATION', 'PAY_0',
       'PAY_2', 'PAY_3', 'BILL_AMT1', 'BILL_AMT2',
       'BILL_AMT3','PayBill1','PayBill2','PayBill3','PayBill4','PayBill5','PayBill6','Default']].corr()

,LIMIT_BAL,MARRIAGE,EDUCATION,PAY_0,PAY_2,PAY_3,BILL_AMT1,BILL_AMT2,BILL_AMT3,PayBill1,PayBill2,PayBill3,PayBill4,PayBill5,PayBill6,Default
LIMIT_BAL,1.000000,-0.108139,-0.219161,-0.271214,-0.296382,-0.286123,0.285430,0.278314,0.283236,-0.007089,-0.021389,-0.041870,-0.059848,-0.052143,-0.050414,-0.153520
MARRIAGE,-0.108139,1.000000,-0.143464,0.019917,0.024199,0.032688,-0.023472,-0.021602,-0.024909,-0.020890,-0.022691,-0.013168,-0.013080,-0.011111,-0.013844,-0.024339
EDUCATION,-0.219161,-0.143464,1.000000,0.105364,0.121566,0.114025,0.023581,0.018749,0.013002,0.026152,0.019478,0.016966,0.020159,0.007296,0.004179,0.028006
PAY_0,-0.271214,0.019917,0.105364,1.000000,0.672164,0.574245,0.187068,0.189859,0.179785,0.487506,0.418755,0.351143,0.317305,0.293100,0.256419,0.324794
PAY_2,-0.296382,0.024199,0.121566,0.672164,1.000000,0.766552,0.234887,0.235257,0.224146,0.387908,0.488559,0.388882,0.341192,0.315568,0.275918,0.263551
PAY_3,-0.286123,0.032688,0.114025,0.574245,0.766552,1.000000,0.208473,0.237295,0.227494,0.298772,0.355999,0.464904,0.372191,0.322261,0.286981,0.235253
BILL_AMT1,0.285430,-0.023472,0.023581,0.187068,0.234887,0.208473,1.000000,0.951484,0.892279,0.282210,0.265612,0.219987,0.170994,0.165299,0.166083,-0.019644
BILL_AMT2,0.278314,-0.021602,0.018749,0.189859,0.235257,0.237295,0.951484,1.000000,0.928326,0.295818,0.256172,0.249528,0.191518,0.183779,0.184468,-0.014193
BILL_AMT3,0.283236,-0.024909,0.013002,0.179785,0.224146,0.227494,0.892279,0.928326,1.000000,0.286169,0.261712,0.192420,0.215957,0.199074,0.192240,-0.014076
PayBill1,-0.007089,-0.020890,0.026152,0.487506,0.387908,0.298772,0.282210,0.295818,0.286169,1.000000,0.814468,0.647730,0.523472,0.468316,0.378763,0.222866


In [11]:
df.drop('ID',axis=1,inplace=True)

In [12]:
df.SEX=df.SEX.map({1:'male',2:'female'})
df.EDUCATION=df.EDUCATION.map({0:'Others0',1:'graduate school',2:'University',3:'High School',4:'Others4',5:'Others5',6:'Others6'})
df.MARRIAGE=df.MARRIAGE.map({0:'Others',1:'married',2:'single',3:'divorce'})

In [13]:
from sklearn.feature_extraction import DictVectorizer

In [14]:
def encode_onehot(df, cols):
    vec = DictVectorizer()
    vec_data = pd.DataFrame(vec.fit_transform(df[cols].to_dict('records')).toarray())
    vec_data.columns = vec.get_feature_names()
    vec_data.index = df.index
    
    df = df.drop(cols, axis=1)
    df = df.join(vec_data)
    return df
X = encode_onehot(df, cols=['SEX'])
X1 =encode_onehot(X, cols=['EDUCATION'])
X2 =encode_onehot(X1, cols=['MARRIAGE'])
X2.columns

Index(['LIMIT_BAL', 'AGE', 'PAY_0', 'PAY_2', 'PAY_3', 'PAY_4', 'PAY_5',
       'PAY_6', 'BILL_AMT1', 'BILL_AMT2', 'BILL_AMT3', 'BILL_AMT4',
       'BILL_AMT5', 'BILL_AMT6', 'PAY_AMT1', 'PAY_AMT2', 'PAY_AMT3',
       'PAY_AMT4', 'PAY_AMT5', 'PAY_AMT6', 'Default', 'PayBill1', 'PayBill2',
       'PayBill3', 'PayBill4', 'PayBill5', 'PayBill6', 'PayPay1', 'PayPay2',
       'PayPay3', 'PayPay4', 'PayPay5', 'PayPay6', 'SEX=female', 'SEX=male',
       'EDUCATION=High School', 'EDUCATION=Others0', 'EDUCATION=Others4',
       'EDUCATION=Others5', 'EDUCATION=Others6', 'EDUCATION=University',
       'EDUCATION=graduate school', 'MARRIAGE=Others', 'MARRIAGE=divorce',
       'MARRIAGE=married', 'MARRIAGE=single'],
      dtype='object')

In [15]:
y=df.Default

In [16]:
X=X2.drop(['Default'],axis=1)

In [17]:
X.shape

(30000, 45)

In [18]:
X[['SEX=female','SEX=male', 'EDUCATION=Others0',
       'EDUCATION=High School',  'EDUCATION=University',
       'EDUCATION=graduate school', 'EDUCATION=Others4', 'EDUCATION=Others5','EDUCATION=Others6','MARRIAGE=married', 'MARRIAGE=Others',
       'MARRIAGE=single', 'MARRIAGE=divorce']].replace(0,-1,inplace=True)


/Users/sibaoxin/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [19]:
scaler = StandardScaler()
scaler.fit(X)
X1 = scaler.transform(X)

/Users/sibaoxin/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/Users/sibaoxin/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  This is separate from the ipykernel package so we can avoid doing imports until


In [20]:
X_train, X_test, y_train, y_test = train_test_split(X1, y, test_size=0.3, random_state=3)

In [24]:
from sklearn.feature_selection import SelectKBest,f_regression

In [29]:
from sklearn import metrics

In [31]:
from sklearn.svm import SVC

In [33]:
X_train, X_test, y_train, y_test = train_test_split(X2, y, test_size=0.3, random_state=3)
modelSVC=SVC()
modelSVC.fit(X_train,y_train)
print(cross_val_score(modelSVC, X_train, y_train))
modelSVC.score(X_train,y_train)

/Users/sibaoxin/anaconda3/lib/python3.6/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/Users/sibaoxin/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
/Users/sibaoxin/anaconda3/lib/python3.6/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/Users/sibaoxin/anaconda3/lib/python3.6/site-packages/sklearn/svm/base.py:19

[0.81316955 0.81228571 0.81768824]


0.8231904761904761

In [35]:
from sklearn.ensemble import RandomForestClassifier

In [36]:
X_train, X_test, y_train, y_test = train_test_split(X2, y, test_size=0.3, random_state=3)
modelRF=RandomForestClassifier()
modelRF.fit(X_train,y_train)
print(cross_val_score(modelRF, X_train, y_train))
modelRF.score(X_train,y_train)

/Users/sibaoxin/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Users/sibaoxin/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


[0.78403085 0.78785714 0.78682669]


0.9567619047619048

In [39]:
from sklearn.ensemble import AdaBoostClassifier

In [40]:
modelAB=AdaBoostClassifier()
modelAB.fit(X_train,y_train)
print(cross_val_score(modelAB, X_train, y_train))
modelAB.score(X_train,y_train)

/Users/sibaoxin/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


[0.81274104 0.81442857 0.81654522]


0.8191428571428572

In [41]:
modelSVC=SVC()
modelSVC.fit(X_train,y_train)
print(cross_val_score(modelSVC, X_train, y_train))
modelSVC.score(X_train,y_train)

/Users/sibaoxin/anaconda3/lib/python3.6/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/Users/sibaoxin/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
/Users/sibaoxin/anaconda3/lib/python3.6/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/Users/sibaoxin/anaconda3/lib/python3.6/site-packages/sklearn/svm/base.py:19

[0.81316955 0.81228571 0.81768824]


0.8231904761904761

In [42]:
modelRF=RandomForestClassifier()
modelRF.fit(X_train,y_train)
print(cross_val_score(modelRF, X_train, y_train))
modelRF.score(X_train,y_train)

/Users/sibaoxin/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Users/sibaoxin/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


[0.78388802 0.78114286 0.78782683]


0.9543809523809523

In [48]:
averageAB = (0.81274104+0.81442857+0.81654522+0.8191428571428572)/4

In [49]:
print(averageAB)

0.8157144217857144


In [50]:
averageSVC = (0.81316955+0.81228571+0.817688240+0.8231904761904761)/4
print(averageSVC)  

0.8165834940476191


In [51]:
predictions = modelSVC.predict(X_test)

In [52]:
Accuracy_Score.append(metrics.accuracy_score(y_test,predictions))
Accuracy_Score

[0.8278888888888889,
 0.8283333333333334,
 0.8272222222222222,
 0.8276666666666667,
 0.8271111111111111,
 0.827,
 0.827,
 0.8265555555555556,
 0.8271111111111111]

In [57]:
Accuracy_Score=[]
for i in range(10,18):
    mod=SelectKBest(score_func=f_regression,k=i)
    X2=mod.fit_transform(X1,y)
    X_train, X_test, y_train, y_test = train_test_split(X2, y, test_size=0.3, random_state=3)
    svc=SVC(C=2)
    svc.fit(X_train,y_train)
    y_pred=svc.predict(X_test)
    Accuracy_Score.append(metrics.accuracy_score(y_test,y_pred))
Accuracy_Score

/Users/sibaoxin/anaconda3/lib/python3.6/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/Users/sibaoxin/anaconda3/lib/python3.6/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/Users/sibaoxin/anaconda3/lib/python3.6/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/Users/sibaoxin/anaconda3/lib/python3.6/site-p

[0.8278888888888889,
 0.8283333333333334,
 0.8272222222222222,
 0.8276666666666667,
 0.8271111111111111,
 0.827,
 0.827,
 0.8265555555555556]

In [59]:
mod=SelectKBest(score_func=f_regression,k=11)
X2=mod.fit_transform(X1,y)
X_train, X_test, y_train, y_test = train_test_split(X2, y, test_size=0.3, random_state=3)
svc=SVC(C=2)
svc.fit(X_train,y_train)
y_pred=svc.predict(X_test)
print(metrics.accuracy_score(y_test,y_pred))

/Users/sibaoxin/anaconda3/lib/python3.6/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


0.8283333333333334


In [60]:
A=[]
for i in range(12,26):
    
    mod=SelectKBest(score_func=f_regression,k=i)
    X2=mod.fit_transform(X1,y)
    X_train, X_test, y_train, y_test = train_test_split(X2, y, test_size=0.3, random_state=3)
    clf = AdaBoostClassifier(n_estimators=90)
    model = clf.fit(X_train,y_train)
    y_predict = model.predict(X_test)
    A.append(metrics.accuracy_score(y_test,y_predict))
A

[0.8277777777777777,
 0.8268888888888889,
 0.8291111111111111,
 0.8287777777777777,
 0.8294444444444444,
 0.8282222222222222,
 0.8292222222222222,
 0.8297777777777777,
 0.8276666666666667,
 0.8288888888888889,
 0.8273333333333334,
 0.8273333333333334,
 0.8288888888888889,
 0.8272222222222222]

In [64]:
mod=SelectKBest(score_func=f_regression,k=19)
X2=mod.fit_transform(X1,y)
X_train, X_test, y_train, y_test = train_test_split(X2, y, test_size=0.3, random_state=3)
A=[]
for i in range(75,95):
    clf = AdaBoostClassifier(n_estimators=i,learning_rate=.1)
    model = clf.fit(X_train,y_train)
    y_predict = model.predict(X_test)
    A.append(metrics.accuracy_score(y_test,y_predict))
    
A

[0.8284444444444444,
 0.8284444444444444,
 0.8283333333333334,
 0.8283333333333334,
 0.8284444444444444,
 0.8284444444444444,
 0.8283333333333334,
 0.8283333333333334,
 0.8283333333333334,
 0.8283333333333334,
 0.8284444444444444,
 0.8284444444444444,
 0.8283333333333334,
 0.8284444444444444,
 0.8285555555555556,
 0.8285555555555556,
 0.8284444444444444,
 0.8284444444444444,
 0.8286666666666667,
 0.8286666666666667]

In [65]:
mod=SelectKBest(score_func=f_regression,k=19)
X2=mod.fit_transform(X1,y)
X_train, X_test, y_train, y_test = train_test_split(X2, y, test_size=0.3, random_state=3)
clf = AdaBoostClassifier(n_estimators=94,learning_rate=.1)
model = clf.fit(X_train,y_train)
y_predict = model.predict(X_test)
accuracy_score= metrics.accuracy_score(y_test,y_predict)
print(accuracy_score)

0.8286666666666667


In [ ]:
# Best model: AdaBoost, with k=19, estimator=94, learning rate =0.1